In [3]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.2 MB/s eta 0:00:00


In [4]:
import keras_tuner

In [5]:
from tensorflow.keras.layers import Dense
from keras_tuner.tuners import RandomSearch

In [6]:
# load the dataset
from tensorflow.keras.datasets import mnist

In [7]:
(X_train,y_train),(X_test,y_test)=mnist.load_data()
X_train=X_train/255.0
X_test=X_test/255.0

11490434/11490434 [==============================] - 0s 0us/step


In [8]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten,Dropout

In [9]:
def build_model(hp):
  model = Sequential()
  model.add(Flatten(input_shape=(28,28)))
  for i in range(hp.Int('num_layers',2,20)): #search for num of layers between 2 to 20
    model.add(Dense(units=hp.Int('units_'+str(i),min_value=32,max_value=512,step=32),activation='relu'))
    #search for number of units
    model.add(Dropout(hp.Float('rate',min_value=0.1, max_value=0.5, step=0.1)))
  model.add(Dense(10,activation='softmax'))
  model.compile(optimizer=Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])), loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  return model


In [17]:
#tuner
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5, #tuner will explore 5 diff set of hyperparameters
                     max_retries_per_trial=2,
                     directory = 'my_dir',
                     project_name='Hyper_tuner')

Reloading Tuner from my_dir/Hyper_tuner/tuner0.json


In [18]:
#demonstration
from keras_tuner import HyperParameters
hp1 = HyperParameters()
#help(hp1)

In [19]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))


Trial 5 Complete [00h 01m 10s]
val_accuracy: 0.11349999904632568

Best val_accuracy So Far: 0.9768000245094299
Total elapsed time: 00h 16m 36s


In [20]:
best_hps=tuner.get_best_hyperparameters()[0]
print(best_hps)

In [21]:
tuner.results_summary()

Results summary
Results in my_dir/Hyper_tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 9
units_0: 480
rate: 0.1
units_1: 192
learning_rate: 0.0001
units_2: 448
units_3: 448
units_4: 448
units_5: 288
units_6: 416
units_7: 480
units_8: 192
units_9: 32
units_10: 256
units_11: 384
units_12: 160
units_13: 512
units_14: 288
Score: 0.9768000245094299

Trial 1 summary
Hyperparameters:
num_layers: 15
units_0: 288
rate: 0.1
units_1: 512
learning_rate: 0.0001
units_2: 64
units_3: 32
units_4: 352
units_5: 64
units_6: 224
units_7: 128
units_8: 224
units_9: 224
units_10: 224
units_11: 32
units_12: 32
units_13: 32
units_14: 32
Score: 0.9537000060081482

Trial 3 summary
Hyperparameters:
num_layers: 14
units_0: 192
rate: 0.2
units_1: 64
learning_rate: 0.0001
units_2: 352
units_3: 352
units_4: 224
units_5: 352
units_6: 416
units_7: 288
units_8: 256
units_9: 32
units_10: 256
units_11: 224
units_12: 192
units_13: 224
units_14: 256

In [22]:
model=tuner.hypermodel.build(best_hps)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_11 (Dense)            (None, 480)               376800    
                                                                 
 dropout_10 (Dropout)        (None, 480)               0         
                                                                 
 dense_12 (Dense)            (None, 192)               92352     
                                                                 
 dropout_11 (Dropout)        (None, 192)               0         
                                                                 
 dense_13 (Dense)            (None, 448)               86464     
                                                                 
 dropout_12 (Dropout)        (None, 448)              

In [23]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
1875/1875 [==============================] - 19s 9ms/step - loss: 0.4542 - accuracy: 0.8506 - val_loss: 0.1480 - val_accuracy: 0.9555
Epoch 2/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1692 - accuracy: 0.9510 - val_loss: 0.1204 - val_accuracy: 0.9616
Epoch 3/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1236 - accuracy: 0.9643 - val_loss: 0.0953 - val_accuracy: 0.9713
Epoch 4/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0993 - accuracy: 0.9713 - val_loss: 0.0878 - val_accuracy: 0.9749
Epoch 5/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0809 - accuracy: 0.9765 - val_loss: 0.0857 - val_accuracy: 0.9766
